# Get PC Information from Local Cure PCs
This script runs the following processes: Copies CPC's SecurityDatabase.mdb from Z:\User to the local machines

## Setup

### Import Libraries

In [ ]:
print("Initializing Python script...\n")
print("Importing libraries...\n")
# Standard Library Imports
import os
import shutil
import sys
from datetime import datetime
import logging
from logging.handlers import RotatingFileHandler
from pathlib import Path

# Third-Party Imports
import pandas as pd
from sqlalchemy import inspect

# Project (Local and Global) Imports
base = Path().resolve().parents[1] # monorepo root
sys.path.insert(0, str(base))
print(base)

from shared_config.src.shared_config.sql import PROD_SERVER as server, PYRO_SERVER as pyro_server, PYRO_DATABASE as database
from shared_utils.src.shared_utils.sql import get_engine, read_table, write_to_sql

### Settings

#### Define Notebook Settings

In [ ]:
# Toggle max rows and max columns
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### Setup Logging

In [ ]:
base = Path().resolve().parents[1]

log_dir = base / 'logs'
log_dir.mkdir(parents=True, exist_ok=True)

log_file = log_dir / f'update_security_databases_{datetime.now():%Y%m%d}.log'

logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s = %(message)s',
    handlers=[
        RotatingFileHandler(log_file, maxBytes=10*1024*1024, backupCount=5),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)

logger.info('=' * 60)
logger.info('START CURE PC AUDIT SCRIPT')
logger.info('=' * 60)
logger.info(f'Start Time: {datetime.now():%Y-%m-%d %H:%M:%S}')
logger.info(f'Log File: {log_file}')

print(f'\nLogging to: {log_file}\n')

#### Set CPC Variables

In [ ]:
cpc_client = r'C\Program Files\CPC Client'
cpc_server = r'C\Program Files\CPC ObjServer'

file = 'SecurityDatabase.mdb'

source = r'\\ut02sa7\ac\User'

### Connect to Pyrometry Database on Production SQL Server

In [ ]:
print(f'Connecting to {server} server {database} database...\n')

engine = get_engine(server, database)

equip_pcs = read_table(engine, 'SELECT * FROM [Equipment PCs]')

inspector = inspect(engine)
inspector.get_table_names()

equip_pcs

## Functions

### Check for SecurityDatabase.mdb

In [ ]:
def find_security_database(pc, ip, pcid, destination_1, destination_2, db_file):
    try:
        security_db = os.path.join('\\' + '\\' + str(ip), destination_1, db_file)
        if os.path.exists(security_db):
            return security_db

        security_db = os.path.join('\\' + '\\' + str(pcid), destination_1, db_file)
        if os.path.exists(security_db):
            return security_db

        security_db = os.path.join('\\' + '\\' + str(ip), destination_2, db_file)
        if os.path.exists(security_db):
            return security_db

        security_db = os.path.join('\\' + '\\' + str(pcid), destination_2, db_file)
        if os.path.exists(security_db):
            return security_db

        print(f'Could not find {db_file} for {pc}: {ip}, {pcid}')
        return None
    except Exception as e:
        print(f'Could not connect to {pc}: {e}')
        return None

In [ ]:
def find_equip_database(equip, site, database_source, file):
    try: 
        if site == 'Dayton':
            return None, None
        
        if site == 'ACCE':
            database = 'ACCE'
            source = rf'{database_source}\{database}\{file}'
            return database, source
        
        elif site == 'G12' or site == 'G14':
            if equip == 'OV52':
                database = 'Oven22'
                source = rf'{database_source}\{database}\{file}'
                return database, source
            else:
                database = 'ACS'
                source = rf'{database_source}\{database}\{file}'
                return database, source
                
        elif site == 'East Gate':
            database = 'EastGate'
            source = rf'{database_source}\{database}\{file}'
            return database, source
            
        elif site == 'FCC':
            if equip == 'OV22':
                database = 'Oven22'
                source = rf'{database_source}\{database}\{file}'
                return database, source
            else:
                database = 'FCC'
                source = rf'{database_source}\{database}\{file}'
                return database, source
                
        elif site == 'H5':
            database = 'IRAD'
            source = rf'{database_source}\{database}\{file}'
            return database, source
            
        elif site == 'HTS':
            database = 'HTS'
            source = rf'{database_source}\{database}\{file}'
            return database, source
            
    except Exception as e:
        print(f'Could not find equip database for {equip}: {e}')
        return None, None

## Run Code

### Extract Info from Local PCs

In [ ]:
logger.info('Starting PC information extraction...')

try:
    # Find SecurityDatabase.mdb
    for index, row in equip_pcs.iterrows():
        pc = row['PC']
        ip = row['IP_Address']
        pcid = row['PCID']
        equip = row['Alt_Name']
        site = row['Site']
        
        security_db = find_security_database(pc, ip, pcid, cpc_client, cpc_server, file)
        database, db_source = find_equip_database(equip, site, source, file)
        if database and source:
            # destination_path = rf'\\ut02sa7\ac\~Cody\Testing\{database}\{equip}'
            destination_path = rf'\\ut02sa7\ac\~Cody\Testing\{database}\{pc}'
            os.makedirs(destination_path, exist_ok=True)
            shutil.copy2(db_source, destination_path)
    
except Exception as e:
    logger.error(f'Error during PC information extraction: {e}')
    raise

## Explore Data

## Output